In [1]:
import pandas as pd
import numpy as np
import math
from tqdm import tqdm as tq

In [2]:
df = pd.read_csv('haberman.csv')

df = df.sample(frac=1).reset_index(drop=True) #Shuffling Dataframe

x_train = df.iloc[ : 50, :] #Slicing Dataframe for Train Data
df_cv = df.iloc[50 : 76, :] #Slicing Dataframe for CV Data
df_test  = df.iloc[76 : , :] #Slicing Dataframe for Test Data

y_train = x_train.iloc[:, -1:]
x_train = x_train.drop('status', axis = 1)

y_cv = df_cv.iloc[:, -1:]
x_cv = df_cv.drop('status', axis = 1)

y_test = df_test.iloc[:, -1:]
x_test = df_test.drop('status', axis = 1)

In [3]:
def convt(inp_data): #Converting Categorical Feature to Binary
    temp = []
    for i in range(len(inp_data)):
        if list(inp_data['status'].values)[i] == 1:
            temp.append(1)
        else:
            temp.append(-1)
    return temp

In [4]:
y_train = convt(y_train)
y_cv = convt(y_cv)
y_test = convt(y_test)

## Stand Scaler

In [5]:
def scal(inpt_data):
    df1 = pd.DataFrame() #Empty DataFrame
    len_col = len(inpt_data.columns.values) #Number of coloumns
    for j in range(len_col): #
        summ = 0
        data = inpt_data.iloc[ : , j:j+1 ].values
        for i in data:
            summ += i # Taking sum of all row in a feature
        mean = summ/(len(data)) #Calculating mean

        diff = 0
        summ = 0
        for i in data:
            diff = (i - mean)**2
            summ += diff
        sd = summ / (len(data)) #Calculating SD
        sd = (sd)**(1/2)

        arr = []
        for i in data:
            arr.append((i - mean) / (sd))
        arr = np.array(arr)
        arr = pd.DataFrame(data = arr)
        df1 = pd.concat([df1, arr], axis = 1) #Appending Scaled Values in Empty DataFrame
    return df1

In [6]:
x_train = scal(x_train)
x_cv = scal(x_cv)
x_test = scal(x_test)

In [7]:
def solver(data, w, lam, y_true): #Using this equation -> https://imgur.com/a/Y98XxNC Eq.1
    data = data.reshape((len(data), 1))
    w_t = w.T
    y_pred, l2, temp, z = [0, 0, 0, 0]
    prod = np.dot(w_t, data)
    z = (y_true * prod)
    l2 = lam * (np.dot(w_t, w)) #Regularization Term
    y_pred = (1.0 +  math.exp(-z))
    y_pred = math.log(y_pred) + l2 #Adding L2 Norm
    return y_pred

In [8]:
def sgd(w, iterr, lr, lam, dataframe, y_true): #By differentiating Eq.1 we get -> https://imgur.com/a/IQzVTVe Eq.2
    for i in tq(range(iterr)):#To avoid math error take absolute equation i.e remove -ve sign from Eq.2 -> https://imgur.com/wAh6OeG
        err_sum = 0
        pred = []
        for j in range(dataframe.shape[0]):
            y_pred = solver(np.array(dataframe.loc[j]), w, lam, y_true[j])
            err = (y_true[j]) - (y_pred)
            err_sum += (err)**2
            for k in range(dataframe.shape[1]):
                num  = (w[k] * y_pred)
                prod = (w[k] * y_pred * dataframe.iloc[j,k])
                deno = (math.exp(prod)) + 1
                w[k] = w[k] - lr * (num/deno)
    return [err_sum, w]

## Tester

In [9]:
iterr = 500
lr = 0.00001
ini_w =  np.random.uniform(size = (x_train.shape[1], 1))
for i in range(1, 31):
    lam = i
    w = sgd(ini_w, iterr, lr, lam, x_train, y_train)
    print( 'When lambda:', lam, 'Error:',w[0][0][0])

  2%|█▎                                                                                | 8/500 [00:00<00:15, 32.41it/s]

When lambda: 1 Error: 152.78302073657267


  1%|▋                                                                                 | 4/500 [00:00<00:13, 36.36it/s]

When lambda: 2 Error: 148.6896747339697


  1%|▋                                                                                 | 4/500 [00:00<00:15, 33.06it/s]

...

When lambda: 28 Error: 44.423513849629344


  1%|▍                                                                                 | 3/500 [00:00<00:18, 27.03it/s]

When lambda: 29 Error: 44.314844887442185


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:15<00:00, 33.29it/s]

When lambda: 30 Error: 44.22021386685363


In [14]:
def tester(X, y_true, lam, w):
    y_pred = []
    for i in range(X.shape[0]):
        y_pred.append(solver(np.array(X.iloc[i]), w, lam, y_true[i]))
    crt = 0 #Counter for correct prediction
    incrt = 0 #Counter for incorrect prediction
    for j in range(len(y_true)):
        if (y_pred[j]) > 0.5: #Creating a ththreshold
            crt += 1
        elif (y_pred[j]) < 0.5:
            incrt += 1
    print('Correct: ', crt, 'Incorrect: ', incrt)
    print('Accuracy: ',crt/(X.shape[0])*100, '%')

In [15]:
tester(x_train, y_train, lam, w[1])

Correct:  50 Incorrect:  0
Accuracy:  100.0 %


In [16]:
tester(x_cv, y_cv, lam, w[1])

Correct:  26 Incorrect:  0
Accuracy:  100.0 %


In [17]:
tester(x_test, y_test, lam, w[1])

Correct:  24 Incorrect:  0
Accuracy:  100.0 %
